## Problem 3: How many people live near shopping centers? (8 points)

In the last step of this analysis, use a *spatial join* to relate data from a population grid data set to the buffer layer created in *problem 2* to find out how many people live in all population grid cells that are **within** 1.5 km distance from each shopping centre. 

Use the same population grid data set as during [lesson 3](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-3/spatial-join.html) (load it directly from WFS, don’t forget to assign a CRS).


*Feel free to divide your solution into more codeblocks than prepared! Remember to add comments to your code :)*

### a) Load the population grid data set and the buffer geometries

Use the same population grid data set as during [lesson 3](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-3/spatial-join.html) (load it directly from WFS, don’t forget to assign a CRS). Load the data into a `GeoDataFrame` called `population_grid`.

(optional) If you want, discard unneeded columns and translate the remaining column names from Finnish to English.

In [1]:
# ADD YOUR OWN CODE HERE
import pathlib 
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data"

In [17]:
import geopandas as gpd
population_grid = gpd.read_file(
    (
        "https://kartta.hsy.fi/geoserver/wfs"
        "?service=wfs"
        "&version=2.0.0"
        "&request=GetFeature"
        "&typeName=asuminen_ja_maankaytto:Vaestotietoruudukko_2020"
        "&srsName=EPSG:3879"
    ),
)
population_grid.crs = crs="EPSG:3879"

In [19]:
population_grid = population_grid[["asukkaita", "geometry"]]
population_grid = population_grid.rename(columns={"asukkaita": "population"})
population_grid

,population,geometry
0,5,"POLYGON ((25472499.995 6685998.998, 25472499.9..."
1,8,"POLYGON ((25472499.995 6684249.004, 25472499.9..."
2,5,"POLYGON ((25472499.995 6683999.005, 25472499.9..."
3,13,"POLYGON ((25472499.995 6682998.998, 25472499.9..."
4,5,"POLYGON ((25472749.993 6690249.003, 25472749.9..."
...,...,...
5832,7,"POLYGON ((25513249.999 6686248.997, 25513249.9..."
5833,6,"POLYGON ((25513249.999 6685998.998, 25513249.9..."
5834,14,"POLYGON ((25513249.999 6685748.999, 25513249.9..."
5835,12,"POLYGON ((25513249.999 6685499.000, 25513249.9..."


In [5]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
import geopandas
import pyproj

assert isinstance(population_grid, geopandas.GeoDataFrame)
assert population_grid.crs == pyproj.CRS("EPSG:3879")



Load the buffers computed in *problem 2* into a `GeoDataFrame` called `shopping_centre_buffers`. Add an `assert` statement to check whether the two data frames are in the same CRS.

In [13]:
# ADD YOUR OWN CODE HERE
shopping_centre_buffers = gpd.read_file(DATA_DIRECTORY / "shopping_centres.gpkg")
assert shopping_centre_buffers.crs == population_grid.crs, "shopping_centre_buffers and population_grid have different CRS"

In [10]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
assert isinstance(shopping_centre_buffers, geopandas.GeoDataFrame)
assert shopping_centre_buffers.geometry.geom_type.unique() == ["Polygon"]
assert shopping_centre_buffers.crs == pyproj.CRS("EPSG:3879")


---

### b) Carry out a *spatial join* between the `population_grid` and the `shopping_centre_buffers`

Join the shopping centre’s `id` column (and others, if you want) to the population grid data frame, for all population grid cells that are **within** the buffer area of each shopping centre. [Use a *join-type* that retains only rows from both input data frames for which the geometric predicate is true](https://geopandas.org/en/stable/gallery/spatial_joins.html#Types-of-spatial-joins). 


In [28]:
# ADD YOUR OWN CODE HERE
population_within_1500_shopping_center = population_grid.sjoin(shopping_centre_buffers, predicate="within")
population_within_1500_shopping_center

,population,geometry,index_right,address,id,name,addr
1147,128,"POLYGON ((25484250.000 6672499.005, 25484250.0...",2,"Iso Omena, 11, Piispansilta, Matinkylä, Suur-M...",2,Iso-omena,"Piispansilta 11, 02230 Espoo, Finland"
1148,81,"POLYGON ((25484250.000 6672249.006, 25484250.0...",2,"Iso Omena, 11, Piispansilta, Matinkylä, Suur-M...",2,Iso-omena,"Piispansilta 11, 02230 Espoo, Finland"
1149,20,"POLYGON ((25484250.000 6671748.997, 25484250.0...",2,"Iso Omena, 11, Piispansilta, Matinkylä, Suur-M...",2,Iso-omena,"Piispansilta 11, 02230 Espoo, Finland"
1211,110,"POLYGON ((25484499.998 6672749.004, 25484499.9...",2,"Iso Omena, 11, Piispansilta, Matinkylä, Suur-M...",2,Iso-omena,"Piispansilta 11, 02230 Espoo, Finland"
1212,136,"POLYGON ((25484499.998 6672499.005, 25484499.9...",2,"Iso Omena, 11, Piispansilta, Matinkylä, Suur-M...",2,Iso-omena,"Piispansilta 11, 02230 Espoo, Finland"
...,...,...,...,...,...,...,...
5287,344,"POLYGON ((25505499.998 6677498.998, 25505499.9...",0,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",0,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland"
5288,340,"POLYGON ((25505499.998 6677248.998, 25505499.9...",0,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",0,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland"
5342,369,"POLYGON ((25505749.995 6677748.997, 25505749.9...",0,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",0,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland"
5343,130,"POLYGON ((25505749.995 6677498.998, 25505749.9...",0,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",0,Itis,"Itäkatu 1-7, 00930 Helsinki, Finland"



---

### c) Compute the population sum around shopping centres

Group the resulting (joint) data frame by shopping centre (`id` or `name`), and calculate the `sum()` of the population living inside the 1.5 km radius around them.

Print the results, for instance, in the form "12345 people live within 1.5 km from REDI".

In [33]:
# ADD YOUR OWN CODE HERE
population_within_1500_shopping_center = population_within_1500_shopping_center.groupby("name").sum()
population_within_1500_shopping_center

,population,index_right,id
name,,,
Forum,57062,76,76
Iso-omena,26698,156,156
Itis,20889,0,0
Jumbo,10907,148,148
REDI,27801,205,205
Sello,22329,189,189
Tripla,24528,318,318


In [45]:
population_within_1500_shopping_center = population_within_1500_shopping_center["population"]

In [47]:
for key, value in population_within_1500_shopping_center.items():
    print(f"{value} people live within 1.5km from {key}")

57062 people live wihtin 1.5km from Forum
26698 people live wihtin 1.5km from Iso-omena
20889 people live wihtin 1.5km from Itis
10907 people live wihtin 1.5km from Jumbo
27801 people live wihtin 1.5km from REDI
22329 people live wihtin 1.5km from Sello
24528 people live wihtin 1.5km from Tripla



---

### d) Reflection

Good job! You are almost done with this week’s exercise. Please quickly answer the following short questions:
    
- How challenging did you find problems 1-3 (on scale to 1-5), and why?
- What was easy?
- What was difficult?

Add your answers in a new *Markdown* cell below:

I rank the difficulities as 3.

It is because the questions do not need us to figure out new algorithm or think outside the scope we are learning. I find the main difficulties are not familar with the syntax, so I have to go through the previous notes to figure out the syntax.